## Authenticate to Google Earth Engine

In [1]:
import ee
from pathlib import Path
import pandas as pd
import ast

ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='raman-461708')

In [2]:
# Import the two AEZ_7_boundaries assets
aez_boundaries_0 = ee.FeatureCollection('projects/raman-461708/assets/AEZ_7_boundaries_0')
aez_boundaries_1 = ee.FeatureCollection('projects/raman-461708/assets/AEZ_7_boundaries_1')

# Merge the two feature collections into one
merged_aez_boundaries = aez_boundaries_0.merge(aez_boundaries_1)

In [3]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("AEZ_7/status.csv")

# Function to create a tile (rectangle polygon) from diagonal points
def create_tile_from_points(points):
    # points is a string representation of a tuple, e.g. "((lat1, lon1), (lat2, lon2))"
    if isinstance(points, str):
        points_tuple = ast.literal_eval(points)
    else:
        points_tuple = points
    (lon1, lat1), (lon2, lat2) = points_tuple
    # Get min/max for lat/lon to define the rectangle
    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)
    # Return the four corners in order (clockwise or counterclockwise)
    return [
        (min_lat, min_lon),
        (min_lat, max_lon),
        (max_lat, max_lon),
        (max_lat, min_lon),
        (min_lat, min_lon)  # close the polygon
    ]

# Create a new column 'tile' with the rectangle polygon for each row
df['tile'] = df['points'].apply(create_tile_from_points)
tiles = df['tile'].tolist()

# Function to create a single tile encompassing four smaller tiles
#def create_group_tile(group):
#    all_points = [point for points in group['tile'] for point in points]
#    lons = [lon for lat, lon in all_points]
#    lats = [lat for lat, lon in all_points]
#    min_lat, max_lat = min(lats), max(lats)
#    min_lon, max_lon = min(lons), max(lons)
#    return [
#        (min_lat, min_lon),
#        (min_lat, max_lon),
#        (max_lat, max_lon),
#        (max_lat, min_lon),
#        (min_lat, min_lon)  # close the polygon
#    ]

# Create a new column 'tile' with the rectangle polygon for each row
#df['tile'] = df['points'].apply(create_tile_from_points)

# Group every four rows and create a single tile encompassing them
#grouped_tiles = []
#for i in range(0, len(df), 4):
#    group_df = df.iloc[i:i+4]
#    grouped_tile = create_group_tile(group_df)
#    grouped_tiles.append(grouped_tile)

#tiles = grouped_tiles

In [4]:
samples = []
import concurrent.futures
import time

# Function to chunk a list into sublists of size p
#def chunk_list(lst, p):
#    return [lst[i:i + p] for i in range(0, len(lst), p)]

# Define the number of tiles per group
#p = 1

# Chunk the tiles into groups
#tile_groups = chunk_list(tiles, p)

def fc_to_df(fc):
    # Get the data from the FeatureCollection
    features = fc.getInfo()['features']
    
    # Extract properties and geometry
    data = [feature['properties'] for feature in features]
    
    return pd.DataFrame(data)

def process_tile(tile, index):
    tile = ee.FeatureCollection(ee.Geometry.Polygon(tile))
    
    #aez_region = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 7)).filter(tile)
    g_embs = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL").filterDate('2024-01-01', '2025-01-01').filterBounds(tile).mosaic()

    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]
    easy_scrubland = [
        ee.Filter.gte("size", 60000)
        ]
    easy_plantation = [
        ee.Filter.lt("area", 20000),
        ee.Filter.gt("area", 1000)
    ]

    all = merged_aez_boundaries.filterBounds(tile)
    farm = all.filter(ee.Filter.And(*easy_farm))
    scrubland = all.filter(ee.Filter.And(easy_scrubland))
    plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))
    mapping = {
        "farm": 1,
        "plantation": 2,
        "scrubland": 3,
        "rest": 0
    }
    reversed_mapping = {v: k for k, v in mapping.items()}
    reversed_ee_mapping = ee.Dictionary(reversed_mapping)

    easy_farm = [
        ee.Filter.gte("rect", 0.67),
        ee.Filter.gt("size", 500),
        ee.Filter.lt("size", 2000),
        ee.Filter.lt("ent", 1)
        ]

    easy_scrubland = [
        ee.Filter.gte("size", 60000)
        ]



    #Filter out farms which doesnot have 3 nearby farms (Removing solo farms inside scrublands)
    farm_buffer = farm.map(lambda x: x.buffer(10))
    farm_image = ee.Image(0)
    farm_mask = farm_image.clip(farm_buffer).mask()
    farm_vectors = farm_mask.toInt().reduceToVectors(
        geometry=farm.geometry(),
        scale=10,  # Change based on your resolution
        geometryType='polygon',
        labelProperty='zone',
        reducer=ee.Reducer.countEvery(),
        maxPixels=1e8
    ) 
    farm_vectors = farm_vectors.filter(ee.Filter.eq('zone', 1)).map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.gt('count', 3))

    #farm_vectors = get_feature_collection("projects/ee-raman/assets/" + block + "_farm_clusters")
    farm = farm.filterBounds(farm_vectors)
    lulc_v3 = ee.Image("projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024").clip(tile.geometry())
    classes_of_interest = [8, 9, 10, 11]
    masked_lulc = lulc_v3.remap(classes_of_interest, [1]*len(classes_of_interest))  # 1 where class is of interest, 0 elsewhere

    # Step 2: Function to compute % area of interest inside each feature
    def filter_by_lulc(feature):
        geom = feature.geometry()
        scale = 30  # Set resolution appropriate to your LULC data

        # Area of interest within polygon (masked_lulc == 1)
        interest_area_img = ee.Image.pixelArea().updateMask(masked_lulc)
        interest_area = interest_area_img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geom,
            scale=scale,
            maxPixels=1e8
        ).get('area')

        # Total area of the polygon
        total_area = ee.Image.pixelArea().reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geom,
            scale=scale,
            maxPixels=1e8
        ).get('area')

        # Compute % and add it as a property
        percent_interest = ee.Number(interest_area).divide(ee.Number(total_area)).multiply(100)
        return feature.set('percent_interest', percent_interest)

    # Step 3: Apply to FeatureCollection
    scrubland = scrubland.map(filter_by_lulc)

    # Step 4: Filter features with > 50% area of interest
    def strictly_inside_roi(feature):
        return feature.set('inside', all.union().geometry().contains(feature.geometry()))

    scrubland = scrubland.filter(ee.Filter.lt('percent_interest', 50)).map(strictly_inside_roi)
    scrubland = scrubland.filter(ee.Filter.eq('inside', True))

    label_image = ee.Image(0).rename("label")
    farm_mask = label_image.clip(farm).mask()
    scrubland_mask = label_image.clip(scrubland).mask()
    plantation_mask = label_image.clip(plantation).mask()

    label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])
    
    # Classes to sample (exclude background = 0)
    class_values = [(1, 150), (2, 150), (3, 150)]

    # Create masks for all classes and combine them into a single mask
    combined_mask = ee.Image(0).rename('class')
    for class_val, points in class_values:
        combined_mask = combined_mask.where(label_image.eq(class_val), class_val)

    # Add the combined mask to the image
    masked_ts = g_embs.addBands(combined_mask.rename('class'))

    # Prepare class values and points for stratified sampling
    class_band = 'class'
    class_values_list = [class_val for class_val, _ in class_values]
    class_points_list = [points for _, points in class_values]

    # Sample uniformly from the masked image using a single call to stratifiedSample
    all_samples = masked_ts.stratifiedSample(
        numPoints=sum(class_points_list),  # Total number of points needed
        classBand=class_band,
        classValues=class_values_list,
        classPoints=class_points_list,
        scale=10,  # Adjust the scale as needed
        region=tile.geometry(),
        seed=42,
        dropNulls=True,
        geometries=False
    )

    # Filter samples to ensure each class has the correct number of points
    def filter_samples(samples, class_val, points):
        return samples.filter(ee.Filter.eq('class', class_val)).limit(points)

    filtered_samples_list = [
        filter_samples(all_samples, class_val, points)
        for class_val, points in class_values
    ]

    # Merge all filtered samples into a single FeatureCollection
    all_filtered_samples = ee.FeatureCollection(filtered_samples_list).flatten()
    task = ee.batch.Export.table.toDrive(
        collection=all_filtered_samples,
        description="tile_" + str(index),
        fileFormat='CSV',
        folder='Scrubland_Field_Delineation',
        fileNamePrefix='tile_' + str(index),
    )
    task.start()
    print(f"tile_" + str(index) + " started")
    while True:
        status = task.status()['state']
        print(f"Task tile_{index} status: {status}")
        if status in ['COMPLETED', 'FAILED', 'CANCELLED']:
            break
        time.sleep(30) 
    #all_filtered_samples_df = fc_to_df(all_filtered_samples)
    #all_filtered_samples_df.to_csv('output_'+str(index)+'.csv', index=False)
    #print("Done tile no ", index)
    

for i in range(2, len(tiles)):
    process_tile(tiles[i], i)

tile_2 started
Task tile_2 status: READY


KeyboardInterrupt: 

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

# Authenticate and create the PyDrive client
gauth = GoogleAuth()
gauth.CommandLineAuth()
drive = GoogleDrive(gauth)

# Folder name in Google Drive
folder_name = 'Scrubland_Field_Delineation'
local_folder = 'Scrubland_Field_Delineation'
os.makedirs(local_folder, exist_ok=True)

# Get folder ID
file_list = drive.ListFile({'q': "mimeType='application/vnd.google-apps.folder' and trashed=false"}).GetList()
folder_id = None
for file in file_list:
    if file['title'] == folder_name:
        folder_id = file['id']
        break

if folder_id is None:
    print("Folder not found!")
else:
    # List all CSV files in the folder
    query = f"'{folder_id}' in parents and trashed=false and mimeType='text/csv'"
    file_list = drive.ListFile({'q': query}).GetList()
    for file in file_list:
        print(f"Downloading {file['title']}")
        file.GetContentFile(os.path.join(local_folder, file['title']))
        
import pandas as pd
import glob

csv_files = glob.glob('Scrubland_Field_Delineation/tile_*.csv')
df_list = [pd.read_csv(f) for f in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)
combined_df.to_csv('Scrubland_Field_Delineation/combined_samples.csv', index=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=903213563690-dc8onmle8ebf0d8qj557hl6ot54snj8q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [ ]:
aez = ee.FeatureCollection("projects/raman-461708/assets/AEZ_7_boundaries_0")
all_samples = ee.FeatureCollection('projects/raman-461708/assets/combined_samples')
classifier = ee.Classifier.smileRandomForest(50).train(
    features=all_samples,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)
classified = ts_data.classify(classifier)

classifier_dynamic = ee.Classifier.smileRandomForest(50).train(
    features=samples_without_plantaiton,
    classProperty='class',
    inputProperties=ts_data.bandNames()
)

task = ee.batch.Export.classifier.toAsset(classifier_dynamic, "Saving dynamic classifier for " + directory, "projects/ee-raman/assets/" + directory + "_classifier")
task.start()
classified_ = ts_data.classify(classifier_dynamic)

,system:index,A00,A01,A02,A03,A04,A05,A06,A07,A08,...,A56,A57,A58,A59,A60,A61,A62,A63,class,.geo
0,0_450,0.022207,-0.044844,0.214133,0.007443,-0.015748,0.029773,0.015748,0.051734,-0.088827,...,-0.079723,0.010396,-0.172795,-0.051734,0.007443,-0.015748,0.027128,0.147697,1,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,0_451,-0.001538,-0.075356,0.206936,-0.048228,-0.059116,0.066990,-0.066990,0.032541,-0.075356,...,-0.035433,-0.066990,-0.153787,-0.079723,-0.044844,-0.015748,0.084214,0.119093,1,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,0_452,0.019931,0.007443,0.172795,0.044844,-0.035433,0.098424,-0.044844,0.006151,-0.079723,...,-0.059116,0.022207,-0.124567,-0.088827,-0.075356,-0.027128,0.141730,0.093564,1,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,0_453,-0.024606,-0.051734,0.192910,0.038447,-0.055363,0.051734,0.071111,0.032541,-0.071111,...,0.017778,-0.017778,-0.221453,-0.103406,0.013841,-0.075356,0.093564,0.093564,1,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,0_454,0.010396,-0.093564,0.172795,0.055363,-0.035433,0.032541,0.044844,0.038447,-0.108512,...,0.010396,0.000984,-0.221453,-0.071111,0.038447,-0.048228,0.084214,0.108512,1,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8902,2_895,0.015748,-0.055363,0.147697,0.098424,-0.055363,-0.007443,-0.079723,0.153787,-0.147697,...,-0.055363,-0.044844,-0.079723,-0.084214,0.010396,-0.088827,0.084214,0.059116,3,"{""type"":""MultiPoint"",""coordinates"":[]}"
8903,2_896,0.055363,-0.032541,0.153787,0.035433,-0.035433,-0.062991,-0.019931,0.166336,-0.084214,...,-0.051734,-0.035433,-0.098424,-0.113741,0.027128,-0.088827,0.055363,0.108512,3,"{""type"":""MultiPoint"",""coordinates"":[]}"
8904,2_897,0.015748,-0.071111,0.135886,0.108512,-0.027128,0.084214,-0.006151,0.079723,0.006151,...,-0.088827,0.051734,-0.013841,-0.038447,-0.041584,-0.098424,0.119093,0.221453,3,"{""type"":""MultiPoint"",""coordinates"":[]}"
8905,2_898,-0.002215,-0.041584,0.124567,0.029773,-0.062991,-0.062991,-0.032541,0.147697,-0.108512,...,-0.071111,-0.066990,-0.071111,-0.098424,-0.004983,-0.119093,0.079723,0.124567,3,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [ ]:
tile = ee.FeatureCollection(ee.Geometry.Polygon(tiles[2]))
index = 2

#aez_region = ee.FeatureCollection("users/mtpictd/agro_eco_regions").filter(ee.Filter.eq("ae_regcode", 7)).filter(tile)
g_embs = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL").filterDate('2024-01-01', '2025-01-01').filterBounds(tile).mosaic()

mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]
easy_scrubland = [
    ee.Filter.gte("size", 60000)
    ]
easy_plantation = [
    ee.Filter.lt("area", 20000),
    ee.Filter.gt("area", 1000)
]

all = merged_aez_boundaries.filterBounds(tile)
farm = all.filter(ee.Filter.And(*easy_farm))
scrubland = all.filter(ee.Filter.And(easy_scrubland))
plantation = all.filter(ee.Filter.eq("class", "plantation")).map(lambda x: x.set("area", x.geometry().area())).filter(ee.Filter.And(easy_plantation))

mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]

easy_scrubland = [
    ee.Filter.gte("size", 60000)
    ]



#Filter out farms which doesnot have 3 nearby farms (Removing solo farms inside scrublands)
farm_buffer = farm.map(lambda x: x.buffer(10))
farm_image = ee.Image(0)
farm_mask = farm_image.clip(farm_buffer).mask()
farm_vectors = farm_mask.toInt().reduceToVectors(
    geometry=farm.geometry(),
    scale=10,  # Change based on your resolution
    geometryType='polygon',
    labelProperty='zone',
    reducer=ee.Reducer.countEvery(),
    maxPixels=1e8
) 
farm_vectors = farm_vectors.filter(ee.Filter.eq('zone', 1)).map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.gt('count', 3))

#farm_vectors = get_feature_collection("projects/ee-raman/assets/" + block + "_farm_clusters")
farm = farm.filterBounds(farm_vectors)
lulc_v3 = ee.Image("projects/corestack-datasets/assets/datasets/LULC_v3_river_basin/pan_india_lulc_v3_2023_2024").clip(tile.geometry())
classes_of_interest = [8, 9, 10, 11]
masked_lulc = lulc_v3.remap(classes_of_interest, [1]*len(classes_of_interest))  # 1 where class is of interest, 0 elsewhere

# Step 2: Function to compute % area of interest inside each feature
def filter_by_lulc(feature):
    geom = feature.geometry()
    scale = 30  # Set resolution appropriate to your LULC data

    # Area of interest within polygon (masked_lulc == 1)
    interest_area_img = ee.Image.pixelArea().updateMask(masked_lulc)
    interest_area = interest_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=scale,
        maxPixels=1e8
    ).get('area')

    # Total area of the polygon
    total_area = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=scale,
        maxPixels=1e8
    ).get('area')

    # Compute % and add it as a property
    percent_interest = ee.Number(interest_area).divide(ee.Number(total_area)).multiply(100)
    return feature.set('percent_interest', percent_interest)

# Step 3: Apply to FeatureCollection
scrubland = scrubland.map(filter_by_lulc)

# Step 4: Filter features with > 50% area of interest
def strictly_inside_roi(feature):
    return feature.set('inside', all.union().geometry().contains(feature.geometry()))

scrubland = scrubland.filter(ee.Filter.lt('percent_interest', 50)).map(strictly_inside_roi)
scrubland = scrubland.filter(ee.Filter.eq('inside', True))

label_image = ee.Image(0).rename("label")
farm_mask = label_image.clip(farm).mask()
scrubland_mask = label_image.clip(scrubland).mask()
plantation_mask = label_image.clip(plantation).mask()

label_image = label_image.where(farm_mask, mapping["farm"]).where(scrubland_mask, mapping["scrubland"]).where(plantation_mask, mapping["plantation"])

# Classes to sample (exclude background = 0)
class_values = [(1, 150), (2, 150), (3, 150)]

# Create masks for all classes and combine them into a single mask
combined_mask = ee.Image(0).rename('class')
for class_val, points in class_values:
    combined_mask = combined_mask.where(label_image.eq(class_val), class_val)

# Add the combined mask to the image
masked_ts = g_embs.addBands(combined_mask.rename('class'))

# Prepare class values and points for stratified sampling
class_band = 'class'
class_values_list = [class_val for class_val, _ in class_values]
class_points_list = [points for _, points in class_values]


masked_ts.
# Merge all filtered samples into a single FeatureCollection
#all_filtered_samples = ee.FeatureCollection(filtered_samples_list).flatten()
#all_filtered_samples_df = fc_to_df(all_filtered_samples)
#all_filtered_samples_df.to_csv('output_'+str(index)+'.csv', index=False)
#print("Done tile no ", index)

SyntaxError: invalid syntax (3624753053.py, line 137)

In [ ]:


# Classes to sample (exclude background = 0)
class_values = [(1,1), (2,1), (3,1)]

# Empty list to store samples
samples_list = []

for class_val, points in class_values:
    # Create a mask for the class
    class_mask = label_image.eq(class_val)
    
    # Mask the ts_image to only include pixels of this class
    masked_ts = g_embs.updateMask(class_mask)
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=tile.geometry(),
            seed=42,
            geometries=True
        )
    
    samples_list.append(class_samples)

all_samples = samples_list[0].merge(samples_list[1]).merge(samples_list[2])
print(f"Total samples collected: {all_samples.size().getInfo()}")

Total samples collected: 3


In [ ]:
# Classes to sample (exclude background = 0)
class_values = [(1, 2000), (2, 2000), (3, 2000)]

# Empty list to store samples
samples_list = []

# Create masks for all classes at once
class_masks = {class_val: label_image.eq(class_val) for class_val, _ in class_values}

# Process each class in parallel
import concurrent.futures

def sample_class(class_val, points):
    # Mask the ts_image to only include pixels of this class
    masked_ts = g_embs.updateMask(class_masks[class_val])
    
    # Sample uniformly from the masked image
    class_samples = masked_ts.addBands(label_image.rename('class')) \
        .stratifiedSample(
            numPoints=points,  # adjust as needed
            classBand='class',
            classValues=[class_val],
            classPoints=[points],  # adjust per class
            scale=10,
            region=tile.geometry(),
            seed=42,
            geometries=True
        )
    return class_samples

with concurrent.futures.ThreadPoolExecutor() as executor:
    samples_list = list(executor.map(lambda cv: sample_class(cv[0], cv[1]), class_values))

# Merge all samples into a single FeatureCollection
all_samples = ee.FeatureCollection(samples_list).flatten()

#print(f"Total samples collected: {all_samples.size().getInfo()}")

In [ ]:
# Classes to sample (exclude background = 0)
class_values = [(1, 2000), (2, 2000), (3, 2000)]

# Create masks for all classes and combine them into a single mask
combined_mask = ee.Image(0).rename('class')
for class_val, points in class_values:
    combined_mask = combined_mask.where(label_image.eq(class_val), class_val)

# Add the combined mask to the image
masked_ts = g_embs.addBands(combined_mask.rename('class'))

# Prepare class values and points for stratified sampling
class_band = 'class'
class_values_list = [class_val for class_val, _ in class_values]
class_points_list = [points for _, points in class_values]

# Sample uniformly from the masked image using a single call to stratifiedSample
all_samples = masked_ts.stratifiedSample(
    numPoints=sum(class_points_list),  # Total number of points needed
    classBand=class_band,
    classValues=class_values_list,
    classPoints=class_points_list,
    scale=10,  # Adjust the scale as needed
    region=tile.geometry(),
    seed=42,
    geometries=True
)

# Filter samples to ensure each class has the correct number of points
def filter_samples(samples, class_val, points):
    return samples.filter(ee.Filter.eq('class', class_val)).limit(points)

filtered_samples_list = [
    filter_samples(all_samples, class_val, points)
    for class_val, points in class_values
]

# Merge all filtered samples into a single FeatureCollection
all_filtered_samples = ee.FeatureCollection(filtered_samples_list).flatten()

#print(f"Total samples collected: {all_filtered_samples.size().getInfo()}")

In [ ]:
all_filtered_samples

{'type': 'FeatureCollection',
 'columns': {'A00': 'Float',
  'A01': 'Float',
  'A02': 'Float',
  'A03': 'Float',
  'A04': 'Float',
  'A05': 'Float',
  'A06': 'Float',
  'A07': 'Float',
  'A08': 'Float',
  'A09': 'Float',
  'A10': 'Float',
  'A11': 'Float',
  'A12': 'Float',
  'A13': 'Float',
  'A14': 'Float',
  'A15': 'Float',
  'A16': 'Float',
  'A17': 'Float',
  'A18': 'Float',
  'A19': 'Float',
  'A20': 'Float',
  'A21': 'Float',
  'A22': 'Float',
  'A23': 'Float',
  'A24': 'Float',
  'A25': 'Float',
  'A26': 'Float',
  'A27': 'Float',
  'A28': 'Float',
  'A29': 'Float',
  'A30': 'Float',
  'A31': 'Float',
  'A32': 'Float',
  'A33': 'Float',
  'A34': 'Float',
  'A35': 'Float',
  'A36': 'Float',
  'A37': 'Float',
  'A38': 'Float',
  'A39': 'Float',
  'A40': 'Float',
  'A41': 'Float',
  'A42': 'Float',
  'A43': 'Float',
  'A44': 'Float',
  'A45': 'Float',
  'A46': 'Float',
  'A47': 'Float',
  'A48': 'Float',
  'A49': 'Float',
  'A50': 'Float',
  'A51': 'Float',
  'A52': 'Float',
  'A5

In [ ]:
classifier = ee.Classifier.smileRandomForest(50).train(
    features=all_samples,
    classProperty='class',
    inputProperties=g_embs.bandNames()
)
classified = g_embs.classify(classifier)

In [ ]:
task = ee.batch.Export.classifier.toAsset(classifier, "Saving dynamic classifier", "projects/raman-461708/assets/aez_7_classifier")
task.start()

In [ ]:
all_samples.limit(1).getInfo()

NameError: name 'all_samples' is not defined